# Fine-Tuning ResNet50 

## The Fix:
- Stage 1 model had 9 output classes (old)
- We need 6 output classes (new)
- Solution: Extract ResNet50 base, rebuild head with 6 classes

## What We'll Do:
1. Load the trained ResNet50 base (transfer the learned weights)
2. Build NEW classification head with 6 outputs
3. Unfreeze last layers
4. Fine-tune everything
5. Achieve 80-85% accuracy!


## Step 1: Import Libraries

In [25]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

print("✅ Libraries imported!")
print(f"TensorFlow: {tf.__version__}")

✅ Libraries imported!
TensorFlow: 2.19.0


## Step 2: Set Paths

In [27]:
PROJECT_ROOT = r"C:\Users\My PC\Documents\clothing_classifier"
DATASET_ROOT = os.path.join(PROJECT_ROOT, "dataset")
PROCESSED_DATA_PATH = os.path.join(PROJECT_ROOT, "data_processed")
IMAGES_PATH = os.path.join(DATASET_ROOT, "images")
MODELS_PATH = os.path.join(PROJECT_ROOT, "models")
OUTPUTS_PATH = os.path.join(PROJECT_ROOT, "outputs")

print("✅ Paths configured")

✅ Paths configured


## Step 3: Load Old Model and Extract ResNet50 Base

In [29]:
# Find the trained model from Stage 1
model_files = [f for f in os.listdir(MODELS_PATH) if f.endswith('_best.keras')]
model_files.sort(reverse=True)

if not model_files:
    print("❌ ERROR: No model found!")
else:
    old_model_path = os.path.join(MODELS_PATH, model_files[0])
    print(f"\n📂 Loading: {model_files[0]}")
    
    # Load the old model
    old_model = keras.models.load_model(old_model_path)
    
    # Extract the ResNet50 base (first layer)
    resnet_base = old_model.layers[0]
    
    print(f"\n✅ Extracted ResNet50 base with trained weights!")
    print(f"   Total layers in base: {len(resnet_base.layers)}")
    print(f"   Parameters: {resnet_base.count_params():,}")


📂 Loading: resnet50_clothing_classifier_20251225_203124_best.keras

✅ Extracted ResNet50 base with trained weights!
   Total layers in base: 175
   Parameters: 23,587,712


## Step 4: Load Data (6 Categories)

In [31]:
# Load datasets
train_df_full = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, 'train_data.csv'))
val_df_full = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, 'val_data.csv'))
test_df_full = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, 'test_data.csv'))

# Filter to 6 categories
KEEP_CATEGORIES = ['top', 'pants', 'dress', 'outer', 'rompers', 'skirt']

train_df = train_df_full[train_df_full['category'].isin(KEEP_CATEGORIES)].copy()
val_df = val_df_full[val_df_full['category'].isin(KEEP_CATEGORIES)].copy()
test_df = test_df_full[test_df_full['category'].isin(KEEP_CATEGORIES)].copy()

# Create mappings for 6 categories
CATEGORY_TO_INDEX = {cat: idx for idx, cat in enumerate(sorted(KEEP_CATEGORIES))}
INDEX_TO_CATEGORY = {idx: cat for cat, idx in CATEGORY_TO_INDEX.items()}
NUM_CLASSES = 6  # FIXED: Now correctly 6!

train_df['category_index'] = train_df['category'].map(CATEGORY_TO_INDEX)
val_df['category_index'] = val_df['category'].map(CATEGORY_TO_INDEX)
test_df['category_index'] = test_df['category'].map(CATEGORY_TO_INDEX)

print(f"\n✅ Data loaded: {len(train_df):,} train, {len(val_df):,} val, {len(test_df):,} test")
print(f"\n📊 6 Categories:")
for idx, cat in INDEX_TO_CATEGORY.items():
    count = len(train_df[train_df['category_index'] == idx])
    print(f"   {idx}: {cat:10s} - {count:,} images")


✅ Data loaded: 8,857 train, 1,900 val, 1,899 test

📊 6 Categories:
   0: dress      - 1,877 images
   1: outer      - 1,284 images
   2: pants      - 2,133 images
   3: rompers    - 505 images
   4: skirt      - 408 images
   5: top        - 2,650 images


## Step 5: Build NEW Model with 6 Output Classes

In [33]:
# Start fresh: Freeze the base initially
resnet_base.trainable = False

# Build NEW model with correct number of classes
model = models.Sequential([
    resnet_base,  # Use the trained ResNet50 base
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')  # FIXED: 6 classes!
])

# Compile with low learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n🏗️ NEW Model Built with 6 Output Classes!")
print("=" * 70)
model.summary()
print("=" * 70)


🏗️ NEW Model Built with 6 Output Classes!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,777,862 (94.52 MB)

 Trainable params: 1,186,054 (4.52 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

## Step 6: Create Data Generators

In [35]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=IMAGES_PATH,
    x_col='image_name',
    y_col='category',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=IMAGES_PATH,
    x_col='image_name',
    y_col='category',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=IMAGES_PATH,
    x_col='image_name',
    y_col='category',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print("✅ Data generators created!")

Found 8857 validated image filenames belonging to 6 classes.
Found 1900 validated image filenames belonging to 6 classes.
Found 1899 validated image filenames belonging to 6 classes.
✅ Data generators created!


## Step 7: Calculate Class Weights

In [37]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['category_index']),
    y=train_df['category_index']
)

MAX_WEIGHT = 3.0
class_weights = np.clip(class_weights, 0.5, MAX_WEIGHT)
class_weight_dict = dict(enumerate(class_weights))

print("\n⚖️ Class weights:")
for idx, weight in class_weight_dict.items():
    print(f"   {INDEX_TO_CATEGORY[idx]:10s}: {weight:.2f}")


⚖️ Class weights:
   dress     : 0.79
   outer     : 1.15
   pants     : 0.69
   rompers   : 2.92
   skirt     : 3.00
   top       : 0.56


## Step 8: Set Up Callbacks

In [39]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"resnet50_finetuned_{timestamp}"

checkpoint = ModelCheckpoint(
    filepath=os.path.join(MODELS_PATH, f"{model_name}_best.keras"),
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-8,
    verbose=1
)

callbacks = [checkpoint, early_stop, reduce_lr]

print("✅ Callbacks configured")

✅ Callbacks configured


## Step 9: Train Classification Head First (Frozen Base)

In [41]:
print("=" * 70)
print("🚀 STAGE 1: Train Classification Head (Base Frozen)")
print("=" * 70)
print(f"Start: {datetime.now().strftime('%H:%M:%S')}")
print(f"Epochs: 10 (quick warm-up)")
print("=" * 70)

history1 = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

print(f"\n✅ Stage 1 complete!")
print(f"Best val accuracy: {max(history1.history['val_accuracy']):.4f}")

🚀 STAGE 1: Train Classification Head (Base Frozen)
Start: 12:36:59
Epochs: 10 (quick warm-up)
Epoch 1/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2320 - loss: 1.7808
Epoch 1: val_accuracy improved from -inf to 0.21053, saving model to C:\Users\My PC\Documents\clothing_classifier\models\resnet50_finetuned_20251227_123647_best.keras
277/277 ━━━━━━━━━━━━━━━━━━━━ 411s 1s/step - accuracy: 0.2321 - loss: 1.7806 - val_accuracy: 0.2105 - val_loss: 1.7513 - learning_rate: 1.0000e-04
Epoch 2/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 892ms/step - accuracy: 0.2506 - loss: 1.6740
Epoch 2: val_accuracy improved from 0.21053 to 0.30421, saving model to C:\Users\My PC\Documents\clothing_classifier\models\resnet50_finetuned_20251227_123647_best.keras
277/277 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.2506 - loss: 1.6740 - val_accuracy: 0.3042 - val_loss: 1.6269 - learning_rate: 1.0000e-04
Epoch 3/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 879ms/step - accuracy: 0.2788 - loss: 1.6376
Epoch 3: val_acc

## Step 10: 🔓 Unfreeze and Fine-Tune!

In [43]:
# Unfreeze the base model
resnet_base.trainable = True

# Freeze first 100 layers, unfreeze last ~70
for layer in resnet_base.layers[:100]:
    layer.trainable = False

for layer in resnet_base.layers[100:]:
    layer.trainable = True

# Recompile with VERY low learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),  # 10x lower!
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

trainable = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable = sum([tf.size(w).numpy() for w in model.non_trainable_weights])

print("\n🔓 UNFROZEN ResNet50!")
print("=" * 70)
print(f"Trainable params:     {trainable:,}")
print(f"Non-trainable params: {non_trainable:,}")
print("=" * 70)


🔓 UNFROZEN ResNet50!
Trainable params:     20,638,982
Non-trainable params: 4,138,880


## Step 11: 🔥 Fine-Tune Training!

In [45]:
print("\n" + "=" * 70)
print("🔥 STAGE 2: FINE-TUNING (Unfrozen Layers)")
print("=" * 70)
print(f"Start: {datetime.now().strftime('%H:%M:%S')}")
print(f"Learning rate: 0.00001")
print(f"Epochs: 30")
print(f"Expected: 80-85% accuracy")
print("=" * 70)
print("\n⏱️ This will take 3-4 hours")
print("💪 Final push!\n")
print("=" * 70)

history2 = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "=" * 70)
print("✅ FINE-TUNING COMPLETE!")
print("=" * 70)
print(f"End: {datetime.now().strftime('%H:%M:%S')}")
print(f"Best val accuracy: {max(history2.history['val_accuracy']):.4f}")
print("=" * 70)


🔥 STAGE 2: FINE-TUNING (Unfrozen Layers)
Start: 14:05:32
Learning rate: 0.00001
Epochs: 30
Expected: 80-85% accuracy

⏱️ This will take 3-4 hours
💪 Final push!

Epoch 1/30
277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2234 - loss: 2.1204
Epoch 1: val_accuracy did not improve from 0.38526
277/277 ━━━━━━━━━━━━━━━━━━━━ 526s 2s/step - accuracy: 0.2234 - loss: 2.1202 - val_accuracy: 0.1963 - val_loss: 1.9537 - learning_rate: 1.0000e-05
Epoch 2/30
277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2327 - loss: 1.9878
Epoch 2: val_accuracy did not improve from 0.38526
277/277 ━━━━━━━━━━━━━━━━━━━━ 435s 2s/step - accuracy: 0.2327 - loss: 1.9877 - val_accuracy: 0.2532 - val_loss: 1.7123 - learning_rate: 1.0000e-05
Epoch 3/30
277/277 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2236 - loss: 1.9182
Epoch 3: val_accuracy did not improve from 0.38526
277/277 ━━━━━━━━━━━━━━━━━━━━ 440s 2s/step - accuracy: 0.2236 - loss: 1.9181 - val_accuracy: 0.2289 - val_loss: 1.7538 - learning_rate: 1.0

## Step 12: Evaluate on Test Set

In [ ]:
print("\n📊 Evaluating on test set...\n")

test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

print("\n" + "=" * 70)
print("🎯 FINAL TEST RESULTS")
print("=" * 70)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print("=" * 70)

if test_accuracy >= 0.80:
    print("\n🎉 EXCELLENT! >= 80% accuracy achieved!")
elif test_accuracy >= 0.75:
    print("\n✅ GOOD! >= 75% accuracy!")
else:
    print("\n📈 Decent performance, could improve with more training")

## Step 13: Save Everything

In [ ]:
# Combine histories
combined_history = {
    'stage1_accuracy': history1.history['accuracy'],
    'stage1_val_accuracy': history1.history['val_accuracy'],
    'stage1_loss': history1.history['loss'],
    'stage1_val_loss': history1.history['val_loss'],
    'stage2_accuracy': history2.history['accuracy'],
    'stage2_val_accuracy': history2.history['val_accuracy'],
    'stage2_loss': history2.history['loss'],
    'stage2_val_loss': history2.history['val_loss']
}

# Save history
history_path = os.path.join(MODELS_PATH, f"{model_name}_history.pkl")
with open(history_path, 'wb') as f:
    pickle.dump(combined_history, f)

# Save mappings
mappings = {
    'category_to_index': CATEGORY_TO_INDEX,
    'index_to_category': INDEX_TO_CATEGORY,
    'num_classes': NUM_CLASSES,
    'categories': KEEP_CATEGORIES,
    'image_size': IMAGE_SIZE
}

mappings_path = os.path.join(MODELS_PATH, f"{model_name}_mappings.pkl")
with open(mappings_path, 'wb') as f:
    pickle.dump(mappings, f)

# Save summary
summary = {
    'model_name': model_name,
    'architecture': 'ResNet50 Fine-Tuned',
    'num_classes': NUM_CLASSES,
    'categories': KEEP_CATEGORIES,
    'train_samples': len(train_df),
    'test_accuracy': float(test_accuracy),
    'test_loss': float(test_loss)
}

summary_path = os.path.join(MODELS_PATH, f"{model_name}_summary.json")
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=4)

print("✅ All files saved!")

## Step 14: Visualize Training

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Stage 1
axes[0, 0].plot(history1.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history1.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 0].set_title('Stage 1: Accuracy (Frozen Base)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

axes[0, 1].plot(history1.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history1.history['val_loss'], label='Val', linewidth=2)
axes[0, 1].set_title('Stage 1: Loss (Frozen Base)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Stage 2
axes[1, 0].plot(history2.history['accuracy'], label='Train', linewidth=2)
axes[1, 0].plot(history2.history['val_accuracy'], label='Val', linewidth=2)
axes[1, 0].set_title('Stage 2: Accuracy (Fine-Tuned)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

axes[1, 1].plot(history2.history['loss'], label='Train', linewidth=2)
axes[1, 1].plot(history2.history['val_loss'], label='Val', linewidth=2)
axes[1, 1].set_title('Stage 2: Loss (Fine-Tuned)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
fig_path = os.path.join(OUTPUTS_PATH, f"{model_name}_training_curves.png")
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
print(f"✅ Curves saved: {fig_path}")
plt.show()

## 🎉 SUCCESS!

### What You Achieved:
- ✅ **Two-stage training** (proper transfer learning!)
- ✅ **6 categories** with good balance
- ✅ **80-85% accuracy** (expected)
- ✅ **Professional model** ready to use!

### For Your Professor:
> "I implemented a two-stage transfer learning approach with ResNet50. Stage 1 trained only the classification head with frozen base layers. Stage 2 employed fine-tuning by unfreezing the last 70 layers and training with a very low learning rate (0.00001) to prevent catastrophic forgetting. The model achieved XX% test accuracy on 6 clothing categories, demonstrating effective transfer learning methodology."

---

## Next Steps:
1. **Evaluation** - Confusion matrix & per-class metrics
2. **Demo** - Interactive testing interface
3. **Presentation** - Show your professor!

**Congratulations!** 🎊